# Ensemble Model

The Ensemble Model used is a combination of Recurrent Neural Network (RNN), Long-Short-Term-Memory Network (LSTM) and Gated Recurrent Unit (GRU) to form a Stacked RNN-LSTM-GRU ensemble model.  

## 1. Import Libraries
Libraries needed for are imported in this step.

In [1]:
import pandas as pd
import numpy as np
import re
import gensim
import string
import nltk
import keras
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.layers import Embedding
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## 2. Read Training Dataset
The training dataset used is the dataset downloaded from Kaggle that contains around 27k rows of Tweets with their sentiments labeled.

In [2]:
# read training dataset
train = pd.read_csv("train.csv", encoding = "unicode_escape")
print("Initial: ")
print(train.head())

Initial: 
       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment Time of Tweet Age of User  \
0  I`d have responded, if I were going   neutral       morning        0-20   
1                             Sooo SAD  negative          noon       21-30   
2                          bullying me  negative         night       31-45   
3                       leave me alone  negative       morning       46-60   
4                        Sons of ****,  negative          noon       60-70   

       Country  Population -2020  Land Area (Km²)  Density (P/Km²)  
0  Afghanistan          38928346         65

## 3. Drop Rows with Missing Values
Rows that contain missing values are dropped to clean the dataset.

In [3]:
# drop rows with null values
train = train.dropna(axis = 0)

## 4. Preprocess Texts
The steps used to preprocess texts are as follows: <br>
a. Remove Links <br>
b. Remove Remove Emails <br>
c. Remove New Line Characters <br>
d. Remove Numbers <br>
e. Remove Emojis <br>
f. Remove Punctuations and Accents <br>
g. Remove Irrelevant Words (Length shorter or equal to 2 OR length greater or equal to 15) <br>
h. Remove Stopwords <br>
i. Tokenization <br>
j. Part-of-Speech (POS) Tagging <br>
k. Lemmatization <br>

In [4]:
stop_words = set(stopwords.words('english'))
def get_wordnet_pos(treebank_pos):
    if treebank_pos.startswith('J'):
        return wordnet.ADJ
    elif treebank_pos.startswith('V'):
        return wordnet.VERB
    elif treebank_pos.startswith('N'):
        return wordnet.NOUN
    elif treebank_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def depure_data(data):
    # Remove URL
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)
    
    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

def sent_to_words(sentences):
    for sentence in sentences:
        # 1. Converts to lowercase
        # 2. Removes accents and punctuations
        # 3. Removes words shorter (<=) or longer (>=) than minimum length of 2 and maximum length of 15
        # 4. Removes numerical characters
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

def stopwords_lemmatize(data):
    # Remove stopwords
    data = " ".join([word for word in str(data).split() if word not in stop_words])
    
    # Apply tokenization
    data = nltk.word_tokenize(data)
    
    # Perform POS tagging
    data = nltk.pos_tag(data)
    
    # Perform lemmatization
    lemmatized_tokens = []

    for i in range(len(data)):
        word, pos = data[i]
        
        lemmatized_tokens.append(nltk.WordNetLemmatizer().lemmatize(word, get_wordnet_pos(pos)))
    
    data = " ".join(lemmatized_tokens)
    
    return data

temp = []
# Convert the values in the dataset to a list
data_to_list = train['text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])

['respond go', 'sooo sad miss san diego', 'bos bully', 'interview leave alone', 'son put release already buy']


## 5. Label Encoding
The label is encoded into 1 for positive, 0 for neutral and -1 for negative before it is one-hot encoded into the following structure: <br>
[neutral, positive, negative]

In [5]:
train['sentiment'] = train['sentiment'].map({"positive": 1, "neutral": 0, "negative": -1})
y = train['sentiment'].to_numpy()
y = keras.utils.to_categorical(y, 3)
print(y[:5])

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


## 6. Feature Extraction
The feature vector or raw word vector is extracted by converting the texts into sequence of numbers. <br>
The maximum number of words that will be stored in the tokenizer is 5000 and the maximum length of the vector is 200, texts that are shorter than the maximum length will be padded with 0s. 

In [6]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...    0 1208    2]
 [   0    0    0 ...   19 1209 1898]
 [   0    0    0 ...    0 1162 3767]
 ...
 [   0    0    0 ...  434  668 2348]
 [   0    0    0 ...    0    0  523]
 [   0    0    0 ...  460  143  312]]


## 7. Train Validation Split
The training dataset is split by 80% and 20% into train and validation set. 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(tweets, y, test_size = 0.2, random_state = 0)

## 8. Model Training
The models in this file are the three individual models, RNN, LSTM, GRU and the ensemble model, Stacked RNN-LSTM-GRU ensemble model. After each model is trained, the model is saved to a .h5 file so that the model can be loaded for prediction without training again and for training the meta-learner. 

### a. RNN

In [57]:
model0 = Sequential()
model0.add(layers.Embedding(max_words, 40, input_length=max_len))
model0.add(layers.SimpleRNN(125, kernel_regularizer=regularizers.l2(l=0.0001)))
model0.add(layers.Dropout(0.8))
model0.add(layers.Dense(10, activation='relu'))
model0.add(layers.Dense(3,activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.0001)
model0.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model0.summary()

checkpoint0 = ModelCheckpoint("best_model0.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model0.fit(X_train, y_train, epochs=8, batch_size=64, validation_data=(X_test, y_test),callbacks=[checkpoint0])
model0.save('RNN.h5')

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 200, 40)           200000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 125)               20750     
                                                                 
 dropout_17 (Dropout)        (None, 125)               0         
                                                                 
 dense_40 (Dense)            (None, 10)                1260      
                                                                 
 dense_41 (Dense)            (None, 3)                 33        
                                                                 
Total params: 222,043
Trainable params: 222,043
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8
344/344 [============================

### b. LSTM

In [61]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 40, input_length=max_len)) 
model1.add(layers.LSTM(20,dropout=0.9)) 
model1.add(layers.Dense(3,activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)
model1.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

checkpoint3 = ModelCheckpoint("best_model3.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=8, batch_size=64, validation_data=(X_test, y_test),callbacks=[checkpoint3])
model1.save('LSTM.h5')

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 200, 40)           200000    
                                                                 
 lstm_6 (LSTM)               (None, 20)                4880      
                                                                 
 dense_42 (Dense)            (None, 3)                 63        
                                                                 
Total params: 204,943
Trainable params: 204,943
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8
343/344 [============================>.] - ETA: 0s - loss: 1.0219 - accuracy: 0.4636
Epoch 1: val_accuracy improved from -inf to 0.62373, saving model to best_model3.hdf5
344/344 [==============================] - 16s 43ms/step - loss: 1.0221 - accuracy: 0.4636 - val_loss: 0.8537 - val_accuracy: 0.62

### c. GRU

In [65]:
model2 = keras.Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len)) 
model2.add(layers.GRU(10, kernel_regularizer=regularizers.l2(l=0.00001)))
model2.add(layers.Dropout(0.9))
model2.add(layers.Dense(10, activation='relu', kernel_regularizer=regularizers.l2(l=0.00001)))
model2.add(layers.Dense(3, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=8, batch_size=64, validation_data=(X_test, y_test),callbacks=[checkpoint2])
model2.save('GRU.h5')

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 200, 40)           200000    
                                                                 
 gru_14 (GRU)                (None, 10)                1560      
                                                                 
 dropout_18 (Dropout)        (None, 10)                0         
                                                                 
 dense_43 (Dense)            (None, 10)                110       
                                                                 
 dense_44 (Dense)            (None, 3)                 33        
                                                                 
Total params: 201,703
Trainable params: 201,703
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8
343/344 [============================

### d. Ensemble Model

i. Function to use the base-learners to predict the probabilities individually and stack the predictions made by the deep learning base-learners before flattening to a 2D array.

In [67]:
def stacked_dataset(members, inputX):
    stackX = None
    itr = 0
    for model in members:
        # predict the outputs using baselearners
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = np.dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

ii. Function to train the SVM meta-learner with the new inputs which are the stacked predictions from the base-learners together with the labels.

In [68]:
def fit_stacked_model_svm(members, inputX, inputy):
    # create dataset that contains the predicted probabilities from the baselearners
    stackedX = stacked_dataset(members, inputX)
    # fit meta-learner
    model = SVC(kernel='poly', C=0.00025)
    model.fit(stackedX, inputy)
    return model

members = []
model = load_model('RNN.h5')
members.append(model)
model = load_model('LSTM.h5')
members.append(model)
model = load_model('GRU.h5')
members.append(model)

sentiment = ['neutral', 'positive', 'negative']
y_train_transform = []
for y in y_train:
    y_train_transform.append(sentiment[np.around(y, decimals=0).argmax()])
# fit stacked model
modelSVM = fit_stacked_model_svm(members, X_train, y_train_transform)

iii. Function to train the LR meta-learner with the new inputs which are the stacked predictions from the base-learners together with the labels.

In [69]:
def fit_stacked_model_lr(members, inputX, inputy):
    # create dataset that contains the predicted probabilities from the baselearners
    stackedX = stacked_dataset(members, inputX)
    # fit meta-learner
    model = LogisticRegression(C=0.001, random_state=0)
    model.fit(stackedX, inputy)
    return model

members = []
model = load_model('RNN.h5')
members.append(model)
model = load_model('LSTM.h5')
members.append(model)
model = load_model('GRU.h5')
members.append(model)

sentiment = ['neutral', 'positive', 'negative']
y_train_transform = []
for y in y_train:
    y_train_transform.append(sentiment[np.around(y, decimals=0).argmax()])
# fit stacked model
modelLR = fit_stacked_model_lr(members, X_train, y_train_transform)

iv. Function to predict the output of data after training the meta-learners.

In [70]:
def stacked_prediction(members, model, inputX):
    # create dataset that contains the predicted probabilities from the baselearners
    stackedX = stacked_dataset(members, inputX)
    # predict the outputs using ensemble model
    yhat = model.predict(stackedX)
    return yhat

## 9. Model Evaluation
Each of the models are evaluated using accuracy, precision, recall and F1-score which are obtained from the classification report. 

### a. RNN

In [60]:
sentiment = ['neutral', 'positive', 'negative']

model0 = load_model('RNN.h5')

print(' RNN MODEL ')
print('===========')

print(' TRAIN ACCURACY ')
print('================')
y_predTrain = model0.predict(X_train)
y_train_transform = []
for y in y_train:
    y_train_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

predicted_train = []
for y in y_predTrain:
    predicted_train.append(sentiment[np.around(y, decimals=0).argmax()])
    
print(classification_report(y_train_transform, predicted_train))


print(" VALIDATION ACCURACY ")
print("=====================")
y_predTest = model0.predict(X_test)
y_test_transform = []
for y in y_test:
    y_test_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

predicted_test = []
for y in y_predTest:
    predicted_test.append(sentiment[np.around(y, decimals=0).argmax()])
    
print(classification_report(y_test_transform, predicted_test))


print(" REVIEW TEST ACCURACY ")
print("======================")
reviewTest = pd.read_csv("iPhoneTest.csv")
X_reviewTest = reviewTest['text']
y_reviewTest = reviewTest['sentiment'].values.tolist()

X_reviewTest = X_reviewTest.values.tolist()
temp = []
for i in range(len(X_reviewTest)):
    temp.append(depure_data(X_reviewTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_reviewTest = tokenizer.texts_to_sequences(data)
X_reviewTest = pad_sequences(X_reviewTest, maxlen=max_len)

y_predictReview = model0.predict(X_reviewTest)
predicted_sentiment_review = []
for y in y_predictReview:
    predicted_sentiment_review.append(sentiment[np.around(y, decimals=0).argmax()])
print(classification_report(y_reviewTest, predicted_sentiment_review))

print(" COVID TEST ACCURACY ")
print("=====================")
covidTest = pd.read_csv("covidTest.csv")
X_covidTest = covidTest['text']
y_covidTest = covidTest['sentiment'].values.tolist()

X_covidTest = X_covidTest.values.tolist()
temp = []
for i in range(len(X_covidTest)):
    temp.append(depure_data(X_covidTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_covidTest = tokenizer.texts_to_sequences(data)
X_covidTest = pad_sequences(X_covidTest, maxlen=max_len)

y_predictCovid = model0.predict(X_covidTest)
predicted_sentiment_covid = []
for y in y_predictCovid:
    predicted_sentiment_covid.append(sentiment[np.around(y, decimals=0).argmax()]) 
print(classification_report(y_covidTest, predicted_sentiment_covid))

 RNN MODEL 
 TRAIN ACCURACY 
687/687 [==============================] - 5s 8ms/step
              precision    recall  f1-score   support

    negative       0.80      0.53      0.64      6258
     neutral       0.63      0.81      0.71      8842
    positive       0.81      0.76      0.78      6884

    accuracy                           0.72     21984
   macro avg       0.75      0.70      0.71     21984
weighted avg       0.73      0.72      0.71     21984

 VALIDATION ACCURACY 
172/172 [==============================] - 1s 8ms/step
              precision    recall  f1-score   support

    negative       0.73      0.46      0.57      1523
     neutral       0.60      0.76      0.67      2275
    positive       0.73      0.71      0.72      1698

    accuracy                           0.66      5496
   macro avg       0.69      0.64      0.65      5496
weighted avg       0.68      0.66      0.66      5496

 REVIEW TEST ACCURACY 
['anyone iphone pro iphone', 'remember first time buy 

### b. LSTM

In [64]:
sentiment = ['neutral', 'positive', 'negative']

model0 = load_model('LSTM.h5')

print(' LSTM MODEL ')
print('============')

print(' TRAIN ACCURACY ')
print('================')
y_predTrain = model0.predict(X_train)
y_train_transform = []
for y in y_train:
    y_train_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

predicted_train = []
for y in y_predTrain:
    predicted_train.append(sentiment[np.around(y, decimals=0).argmax()])
    
print(classification_report(y_train_transform, predicted_train))


print(" VALIDATION ACCURACY ")
print("=====================")
y_predTest = model0.predict(X_test)
y_test_transform = []
for y in y_test:
    y_test_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

predicted_test = []
for y in y_predTest:
    predicted_test.append(sentiment[np.around(y, decimals=0).argmax()])
    
print(classification_report(y_test_transform, predicted_test))


print(" REVIEW TEST ACCURACY ")
print("======================")
reviewTest = pd.read_csv("iPhoneTest.csv")
X_reviewTest = reviewTest['text']
y_reviewTest = reviewTest['sentiment'].values.tolist()

X_reviewTest = X_reviewTest.values.tolist()
temp = []
for i in range(len(X_reviewTest)):
    temp.append(depure_data(X_reviewTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_reviewTest = tokenizer.texts_to_sequences(data)
X_reviewTest = pad_sequences(X_reviewTest, maxlen=max_len)

y_predictReview = model0.predict(X_reviewTest)
predicted_sentiment_review = []
for y in y_predictReview:
    predicted_sentiment_review.append(sentiment[np.around(y, decimals=0).argmax()])
print(classification_report(y_reviewTest, predicted_sentiment_review))

print(" COVID TEST ACCURACY ")
print("=====================")
covidTest = pd.read_csv("covidTest.csv")
X_covidTest = covidTest['text']
y_covidTest = covidTest['sentiment'].values.tolist()

X_covidTest = X_covidTest.values.tolist()
temp = []
for i in range(len(X_covidTest)):
    temp.append(depure_data(X_covidTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_covidTest = tokenizer.texts_to_sequences(data)
X_covidTest = pad_sequences(X_covidTest, maxlen=max_len)

y_predictCovid = model0.predict(X_covidTest)
predicted_sentiment_covid = []
for y in y_predictCovid:
    predicted_sentiment_covid.append(sentiment[np.around(y, decimals=0).argmax()]) 
print(classification_report(y_covidTest, predicted_sentiment_covid))

 LSTM MODEL 
 TRAIN ACCURACY 
687/687 [==============================] - 6s 8ms/step
              precision    recall  f1-score   support

    negative       0.82      0.68      0.74      6258
     neutral       0.70      0.81      0.75      8842
    positive       0.82      0.77      0.80      6884

    accuracy                           0.76     21984
   macro avg       0.78      0.76      0.76     21984
weighted avg       0.77      0.76      0.76     21984

 VALIDATION ACCURACY 
172/172 [==============================] - 2s 8ms/step
              precision    recall  f1-score   support

    negative       0.76      0.61      0.68      1523
     neutral       0.66      0.77      0.71      2275
    positive       0.78      0.73      0.76      1698

    accuracy                           0.72      5496
   macro avg       0.73      0.71      0.71      5496
weighted avg       0.72      0.72      0.71      5496

 REVIEW TEST ACCURACY 
['anyone iphone pro iphone', 'remember first time buy

### c. GRU

In [49]:
sentiment = ['neutral', 'positive', 'negative']

model0 = load_model('GRU.h5')

print(' GRU MODEL ')
print('===========')

print(' TRAIN ACCURACY ')
print('================')
y_predTrain = model0.predict(X_train)
y_train_transform = []
for y in y_train:
    y_train_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

predicted_train = []
for y in y_predTrain:
    predicted_train.append(sentiment[np.around(y, decimals=0).argmax()])
    
print(classification_report(y_train_transform, predicted_train))


print(" VALIDATION ACCURACY ")
print("=====================")
y_predTest = model0.predict(X_test)
y_test_transform = []
for y in y_test:
    y_test_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

predicted_test = []
for y in y_predTest:
    predicted_test.append(sentiment[np.around(y, decimals=0).argmax()])
    
print(classification_report(y_test_transform, predicted_test))


print(" REVIEW TEST ACCURACY ")
print("======================")
reviewTest = pd.read_csv("iPhoneTest.csv")
X_reviewTest = reviewTest['text']
y_reviewTest = reviewTest['sentiment'].values.tolist()

X_reviewTest = X_reviewTest.values.tolist()
temp = []
for i in range(len(X_reviewTest)):
    temp.append(depure_data(X_reviewTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_reviewTest = tokenizer.texts_to_sequences(data)
X_reviewTest = pad_sequences(X_reviewTest, maxlen=max_len)

y_predictReview = model0.predict(X_reviewTest)
predicted_sentiment_review = []
for y in y_predictReview:
    predicted_sentiment_review.append(sentiment[np.around(y, decimals=0).argmax()])
print(classification_report(y_reviewTest, predicted_sentiment_review))

print(" COVID TEST ACCURACY ")
print("=====================")
covidTest = pd.read_csv("covidTest.csv")
X_covidTest = covidTest['text']
y_covidTest = covidTest['sentiment'].values.tolist()

X_covidTest = X_covidTest.values.tolist()
temp = []
for i in range(len(X_covidTest)):
    temp.append(depure_data(X_covidTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_covidTest = tokenizer.texts_to_sequences(data)
X_covidTest = pad_sequences(X_covidTest, maxlen=max_len)

y_predictCovid = model0.predict(X_covidTest)
predicted_sentiment_covid = []
for y in y_predictCovid:
    predicted_sentiment_covid.append(sentiment[np.around(y, decimals=0).argmax()]) 
print(classification_report(y_covidTest, predicted_sentiment_covid))

 GRU MODEL 
 TRAIN ACCURACY 
687/687 [==============================] - 5s 6ms/step
              precision    recall  f1-score   support

    negative       0.91      0.36      0.52      6258
     neutral       0.60      0.92      0.73      8842
    positive       0.89      0.77      0.83      6884

    accuracy                           0.72     21984
   macro avg       0.80      0.69      0.69     21984
weighted avg       0.78      0.72      0.70     21984

 VALIDATION ACCURACY 
172/172 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

    negative       0.87      0.32      0.47      1523
     neutral       0.57      0.87      0.69      2275
    positive       0.79      0.68      0.73      1698

    accuracy                           0.66      5496
   macro avg       0.74      0.62      0.63      5496
weighted avg       0.72      0.66      0.64      5496

 REVIEW TEST ACCURACY 
['anyone iphone pro iphone', 'remember first time buy 

### d. Stacked RNN-LSTM-GRU with SVM meta-learner

In [71]:
sentiment = ['neutral', 'positive', 'negative']

print(' ENSEMBLE MODEL WITH SVM ')
print('=========================')
print(' TRAIN SET ACCURACY ')
print('====================')
y_train_transform = []
for y in y_train:
    y_train_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

yhat = stacked_prediction(members, modelSVM, X_train)

print(classification_report(y_train_transform, yhat))


print(' VALIDATION SET ACCURACY ')
print('=========================')

y_test_transform = []
for y in y_test:
    y_test_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

yhat = stacked_prediction(members, modelSVM, X_test)

print(classification_report(y_test_transform, yhat))


print(" REVIEW TEST ACCURACY ")
print("======================")
reviewTest = pd.read_csv("iPhoneTest.csv")
X_reviewTest = reviewTest['text']
y_reviewTest = reviewTest['sentiment'].values.tolist()

X_reviewTest = X_reviewTest.values.tolist()
temp = []
for i in range(len(X_reviewTest)):
    temp.append(depure_data(X_reviewTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_reviewTest = tokenizer.texts_to_sequences(data)
X_reviewTest = pad_sequences(X_reviewTest, maxlen=max_len)

y_predictReview = stacked_prediction(members, modelSVM, X_reviewTest)

print(classification_report(y_reviewTest, y_predictReview))


print(" COVID TEST ACCURACY ")
print("=====================")
covidTest = pd.read_csv("covidTest.csv")
X_covidTest = covidTest['text']
y_covidTest = covidTest['sentiment'].values.tolist()

X_covidTest = X_covidTest.values.tolist()
temp = []
for i in range(len(X_covidTest)):
    temp.append(depure_data(X_covidTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_covidTest = tokenizer.texts_to_sequences(data)
X_covidTest = pad_sequences(X_covidTest, maxlen=max_len)

y_predictCovid = stacked_prediction(members, modelSVM, X_covidTest)

print(classification_report(y_covidTest, y_predictCovid))

 ENSEMBLE MODEL WITH SVM 
 TRAIN SET ACCURACY 
              precision    recall  f1-score   support

    negative       0.82      0.68      0.74      6258
     neutral       0.71      0.82      0.76      8842
    positive       0.84      0.81      0.82      6884

    accuracy                           0.78     21984
   macro avg       0.79      0.77      0.78     21984
weighted avg       0.78      0.78      0.78     21984

 VALIDATION SET ACCURACY 
              precision    recall  f1-score   support

    negative       0.76      0.60      0.67      1523
     neutral       0.66      0.77      0.71      2275
    positive       0.77      0.75      0.76      1698

    accuracy                           0.71      5496
   macro avg       0.73      0.71      0.71      5496
weighted avg       0.72      0.71      0.71      5496

 REVIEW TEST ACCURACY 
['anyone iphone pro iphone', 'remember first time buy smart phone iphone show grandfather proud could first question able make rich try prove 

### e. Stacked RNN-LSTM-GRU with LR meta-learner

In [72]:
sentiment = ['neutral', 'positive', 'negative']

print(' ENSEMBLE MODEL WITH LR ')
print('========================')
print(' TRAIN SET ACCURACY ')
print('====================')
y_train_transform = []
for y in y_train:
    y_train_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

yhat = stacked_prediction(members, modelLR, X_train)

print(classification_report(y_train_transform, yhat))


print(' VALIDATION SET ACCURACY ')
print('=========================')
y_test_transform = []
for y in y_test:
    y_test_transform.append(sentiment[np.around(y, decimals = 0).argmax()])

yhat = stacked_prediction(members, modelLR, X_test)

print(classification_report(y_test_transform, yhat))


print(" REVIEW TEST ACCURACY ")
print("======================")
reviewTest = pd.read_csv("iPhoneTest.csv")
X_reviewTest = reviewTest['text']
y_reviewTest = reviewTest['sentiment'].values.tolist()

X_reviewTest = X_reviewTest.values.tolist()
temp = []
for i in range(len(X_reviewTest)):
    temp.append(depure_data(X_reviewTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_reviewTest = tokenizer.texts_to_sequences(data)
X_reviewTest = pad_sequences(X_reviewTest, maxlen=max_len)

y_predictReview = stacked_prediction(members, modelLR, X_reviewTest)

print(classification_report(y_reviewTest, y_predictReview))


print(" COVID TEST ACCURACY ")
print("=====================")
covidTest = pd.read_csv("covidTest.csv")
X_covidTest = covidTest['text']
y_covidTest = covidTest['sentiment'].values.tolist()

X_covidTest = X_covidTest.values.tolist()
temp = []
for i in range(len(X_covidTest)):
    temp.append(depure_data(X_covidTest[i]))
data_words = list(sent_to_words(temp))
detokenized = []
for i in range(len(data_words)):
    detokenized.append(detokenize(data_words[i]))
data = []
for i in range(len(detokenized)):
    data.append(stopwords_lemmatize(detokenized[i]))
print(data[:5])
X_covidTest = tokenizer.texts_to_sequences(data)
X_covidTest = pad_sequences(X_covidTest, maxlen=max_len)

y_predictCovid = stacked_prediction(members, modelLR, X_covidTest)

print(classification_report(y_covidTest, y_predictCovid))

 ENSEMBLE MODEL WITH LR 
 TRAIN SET ACCURACY 
              precision    recall  f1-score   support

    negative       0.81      0.67      0.74      6258
     neutral       0.70      0.81      0.76      8842
    positive       0.83      0.80      0.82      6884

    accuracy                           0.77     21984
   macro avg       0.78      0.76      0.77     21984
weighted avg       0.78      0.77      0.77     21984

 VALIDATION SET ACCURACY 
              precision    recall  f1-score   support

    negative       0.75      0.60      0.67      1523
     neutral       0.66      0.76      0.71      2275
    positive       0.76      0.75      0.76      1698

    accuracy                           0.71      5496
   macro avg       0.73      0.70      0.71      5496
weighted avg       0.72      0.71      0.71      5496

 REVIEW TEST ACCURACY 
['anyone iphone pro iphone', 'remember first time buy smart phone iphone show grandfather proud could first question able make rich try prove r